# Setup Environment

In [ ]:
# @title Install apt dependencies
!apt-get update
!apt-get install git help2man perl python3 make autoconf g++ flex bison ccache libgoogle-perftools-dev numactl perl-doc libfl2 libfl2 zlib1g zlib1g-dev
### Installing pip dependencies
!pip install openai==0.28
!pip install gdown
MODEL="gpt-3.5-turbo-16k"

In [ ]:
# @title Verilator Setup
import gdown
url = 'https://drive.google.com/uc?id=1BvwwX_TTbqZDi0eaLP_I52CDm3gnnSf8'
output = 'verilator.zip'
gdown.download(url, output, quiet=False)
!unzip verilator.zip
!chmod -R +x /content/verilator/bin
%env VERILATOR_ROOT=/content/verilator
%env PATH=/content/verilator/bin:/opt/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin

## Final Setup Step
Load the rest of the required files for the demo and set them up:

In [ ]:
!git clone https://github.com/tamu-seth-lab-soc/llm_assertions.git

## Check the Setup
Check the setup by simulating a simple verilog code:
- The simulation should output `Welcome to the tutorial on generating assertions using LLMs!`

In [ ]:
%cd  /content/llm_assertions/sim
!verilator --binary -j 0 ../src/helloworld.sv 1> comp.log 2>&1
!obj_dir/Vhelloworld

# Task 1: Traffic controller design

**Goal:** Understand the functionality of traffic controller, simulate it with a simple input, and check the output
- You can check the source code of the traffic controller here: `/content/llm_assertions/src/traffic_controller.sv`
- Below is an abstract diagram explaining the code:

traffic_controller.svg

In [ ]:
%cd  /content/llm_assertions/sim
!verilator --assert --binary -j 0  ../tb/tb_simple.sv ../src/traffic_controller.sv 1> comp.log 2>&1
!obj_dir/Vtb_simple

# Task 2: Generate assertions using LLMs
**Goal:** Understand the prompt for LLM, run the LLM, check the response
- The subsequent code queries LLM with different levels of details in the prompt to generate the assertion

In [ ]:
#@title Utility functions

import openai

class ChatGPT3p5():
    """ChatGPT Large Language Model."""

    def __init__(self):
        openai.api_key=""

    def generate(self, messages, seed=None,temperature=0.7):
        response = openai.ChatCompletion.create(
            model=MODEL,
            messages=messages,
            seed=seed, # For deterministic outputs
            temperature=temperature  # For deterministic outputs
        )
        return response['choices'][0]['message']['content']

In [ ]:
#@title Simple prompt for YELLOW->RED with no context

prompt = "Generate a hardware assertion to check that a traffic signal\
becomes red after yellow"
model = ChatGPT3p5() # <-- we are using ChatGPT 3.5
messages=[ # <-- this is the prompt to LLM
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

print(model.generate(messages)) # <-- this prints the response from the LLM

In [ ]:
#@title Detailed prompt for YELLOW->RED with assertion beginning

prompt = "Complete the following hardware assertion to check that a traffic \
#signal named 'signal''s value is 'RED' after 'YELLOW'\nassert property "

model = ChatGPT3p5() # <-- we are using ChatGPT 3.5
messages=[ # <-- this is the prompt to LLM
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

print(model.generate(messages)) # <-- this prints the response from the LLM

In [ ]:
#@title Prompt for YELLOW->RED with design source as input

with open("/content/llm_assertions/src/traffic_controller.sv",'r') as f:
    design_source = f.read()
prompt = design_source
prompt += "Complete the following hardware assertion to check at every positive edge of the clock that \
 if the value of the signal is YELLOW, then it is RED in the next cycle\nassert property "

model = ChatGPT3p5() # <-- we are using ChatGPT 3.5
messages=[ # <-- this is the prompt to LLM
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

print(model.generate(messages)) # <-- this prints the response from the LLM

In [ ]:
#@title Very detailed prompt for YELLOW->RED with detailed example hardware assertion

prompt = "An example of hardware assertion to check that at every \
positive edge of clock, if the value of 'data' is VALID, \
then the value of 'data' in the next clock cycle is DONE is: \
assert property (@(posedge clk) (data == VALID) |=> (data == DONE));\n"

prompt += "Using the above example, generate a hardware assertion to check that at \
every positive edge of clk, if the value of 'signal' is YELLOW, then \
the value of 'signal' in the next clock cycle is RED. "

model = ChatGPT3p5() # <-- we are using ChatGPT 3.5
messages=[ # <-- this is the prompt to LLM
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

print(model.generate(messages)) # <-- this prints the response from the LLM
#print(model.generate(messages,seed=12345,temperature=0.0)) # <-- this tries to generate a deterministic response from the LLM

**Goal:** Simulate the traffic controller with the LLM-generated assertions and evaluate the correctness of the assertion based on simulation output
- Since the assertions generated by the LLM may be incorrect, we compile the testbench and simulate it seperately as shown in the next two commands
- For the LLM-generated assertion do the following to evalaute it:
  - (This is a manual step) Open the testbench located at /content/llm_assertions/tb/tb_triggering.sv and add the LLM-generated assertion at line 130.
  - Run the compile command and check the `/content/llm_assertions/sim/comp.log` file to make sure there are no compilation errors
  - If the compilation failed, there is no need to run the simulation. If the compilation succeeded, run the simulation command

In [ ]:
#@title Compile
%cd  /content/llm_assertions/sim
!verilator --assert --binary -j 0  ../tb/tb_simple.sv ../src/traffic_controller.sv 1> comp.log 2>&1
#!verilator --assert --binary -j 0  ../tb/tb_triggering.sv ../src/traffic_controller.sv ../src/traffic_controller_golden.sv  1> comp.log 2>&1

In [ ]:
#@title Simulate
%cd  /content/llm_assertions/sim
!obj_dir/Vtb_simple
#!obj_dir/Vtb_triggering

# Task 3: Traffic controller bug
**Goal:** Understand the bug in traffic controller, simulate it with bug triggering input and check the output
- To highlight the bug in traffic controller, this testbench simulates both the buggy traffic controller and a bugfree golden traffic controller and compares their outputs
- Notice the time instance where `signal_diff`'s value is `1` in the simulation output. These are instances where the output of the buggy traffic controller is different from that of bugfree golden traffic controller

In [ ]:
%cd  /content/llm_assertions/sim
!verilator --assert --binary -j 0  ../tb/tb_triggering.sv ../src/traffic_controller.sv ../src/traffic_controller_golden.sv  1> comp.log 2>&1
!obj_dir/Vtb_triggering

# Task 4: Finding the bug using LLM
**Goal:** Generate assertion using LLM, compile, simulate, find the bug

Recall that the bug is the transition highlighted in red below:

traffic_controller_bug_highlighted.svg

In [ ]:
#@title Finding the bug

prompt = "Complete the following hardware assertion to check that at \
every positive edge of clk, if the value of the signal register is RED, then \
the value of the signal register in the previous clock cycle is either RED or\
 YELLOW\nassert property. "

prompt += "An example of hardware assertion to check that at every \
positive edge of clock, if the value of the data register is DONE, \
then the value of the data register in the previous clock cycle is either VALID\
 or DONE is: assert property (@(posedge clk) (data == DONE) |-> ($past(data) == VALID) || ($past(data) == DONE));"

model = ChatGPT3p5() # <-- we are using ChatGPT 3.5
messages=[ # <-- this is the prompt to LLM
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ]

print(model.generate(messages)) # <-- this prints the response from the LLM
#print(model.generate(messages,seed=12345,temperature=0.0)) # <-- this tries to generate a deterministic response from the LLM

**Next**
- (This is a manual step) Open the testbench located at /content/llm_assertions/tb/tb_triggering.sv and add the LLM-generated assertion at line 131.
- Run the compile command and check the /content/llm_assertions/sim/comp.log file to make sure there are no compilation errors
- If the compilation failed, there is no need to run the simulation. If the compilation succeeded, run the simulation command

In [ ]:
#@title Compile
%cd  /content/llm_assertions/sim
!verilator --assert --binary -j 0  ../tb/tb_triggering.sv ../src/traffic_controller.sv ../src/traffic_controller_golden.sv  1> comp.log 2>&1

In [ ]:
#@title Simulate
%cd  /content/llm_assertions/sim
!obj_dir/Vtb_triggering

The error message above shows that the assertion was triggered and the bug was found